In [1]:
import torch
from transformers import AutoTokenizer, AutoModel

/Users/nikhileshsirohi/.pyenv/versions/3.10.14/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [3]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

In [4]:
texts = ["Guidelines for PM-Surya Ghar: Muft Bijli Yojana Central Financial Assistance to Residential Consumers Annexure 2 Model Draft Agreement between Consumer & Vendor for installation of grid connected rooftop solar (RTS) project under PM – Surya Ghar: Muft Bijli Yojana", "Model Draft Agreement between Consumer & Vendor for installation of grid connected rooftop solar (RTS) project under PM – Surya Ghar: Muft Bijli Yojana This agreement is executed on -------(Day)------(Month)-------(Year) for design, supply, installation, commissioning and 5 -year comprehensive maintenance of RTS project/system along with warranty under PM Surya Ghar: Muft Bijli Yojana"]

In [5]:
encoded = tokenizer(
    texts,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

In [6]:
encoded = {k: v.to(device) for k, v in encoded.items()}

In [7]:
with torch.no_grad():
    model_output = model(**encoded)

In [8]:
token_embeddings = model_output.last_hidden_state
token_embeddings.shape

torch.Size([2, 108, 384])

In [9]:
attention_mask = encoded["attention_mask"].unsqueeze(-1)
attention_mask.shape

torch.Size([2, 108, 1])

In [10]:
embeddings = (token_embeddings * attention_mask).sum(dim=1)


In [39]:
print(attention_mask.sum(dim=1))

tensor([[ 61],
        [108]], device='mps:0')


In [11]:
embeddings = embeddings / attention_mask.sum(dim=1)

# FAISS

In [12]:
import faiss
import numpy as np

In [15]:
chunk_embeddings = embeddings.to("cpu").numpy()
chunk_embeddings.shape

(2, 384)

In [18]:
chunk_embeddings[0][:10]

array([-0.00081556,  0.1762124 ,  0.03424087, -0.02070345,  0.02350667,
       -0.06562926,  0.05372025,  0.11142264, -0.15108058, -0.04727024],
      dtype=float32)

In [20]:
index = faiss.IndexFlatIP(chunk_embeddings.shape[1])
index

<faiss.swigfaiss.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x3161cbcf0> >

In [21]:
chunks = []

In [22]:
faiss.normalize_L2(chunk_embeddings)
print(chunk_embeddings[0][:10])

[-0.00031852  0.0688213   0.01337307 -0.00808591  0.00918074 -0.02563209
  0.02098092  0.04351709 -0.05900585 -0.01846181]


In [24]:
index.add(chunk_embeddings)
chunks.extend(["Guidelines for PM-Surya Ghar: Muft Bijli Yojana Central Financial Assistance to Residential Consumers Annexure 2 Model Draft Agreement between Consumer & Vendor for installation of grid connected rooftop solar (RTS) project under PM – Surya Ghar: Muft Bijli Yojana", "Model Draft Agreement between Consumer & Vendor for installation of grid connected rooftop solar (RTS) project under PM – Surya Ghar: Muft Bijli Yojana This agreement is executed on -------(Day)------(Month)-------(Year) for design, supply, installation, commissioning and 5 -year comprehensive maintenance of RTS project/system along with warranty under PM Surya Ghar: Muft Bijli Yojana"])
chunks

['Guidelines for PM-Surya Ghar: Muft Bijli Yojana Central Financial Assistance to Residential Consumers Annexure 2 Model Draft Agreement between Consumer & Vendor for installation of grid connected rooftop solar (RTS) project under PM – Surya Ghar: Muft Bijli Yojana',
 'Model Draft Agreement between Consumer & Vendor for installation of grid connected rooftop solar (RTS) project under PM – Surya Ghar: Muft Bijli Yojana This agreement is executed on -------(Day)------(Month)-------(Year) for design, supply, installation, commissioning and 5 -year comprehensive maintenance of RTS project/system along with warranty under PM Surya Ghar: Muft Bijli Yojana']

In [25]:
query = "What is the Guideline?"

In [ ]:
query_embedding = embed_texts([query]).numpy()